In [41]:
import requests
import os
import json
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd

In [2]:
download_location = "C:/Users/Lara/Work/DataDownloads/testData/"
temp_download_location = "C:/Users/Lara/Work/DataDownloads/testDataZip/"
lookup_location = "C:/Users/Lara/Work/DataDownloads/Lookups/"
item_url_file = "ItemURL_ArcGIS_6monthUpdates.csv"

In [3]:
# Read the csv file
item_url_df = pd.read_csv(lookup_location + item_url_file)
item_url_df.head(3)

,Dataset,Owner,URL
0,Local Nature Reserves (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
1,National Nature Reserves (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
2,Priority Habitats Inventory (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...


In [ ]:
#Loop through each dataset name in lookup
for url in item_url_df.URL:
    print(url)
    # Get details for dataset
    url_details = url.replace("/query?", "?f=json")
    
    dataset = item_url_df[item_url_df.URL==url]['Dataset'].item() 
    source = item_url_df[item_url_df.URL==url]['Owner'].item()

    # DOwnload dataset
    params = {
    'where': "1=1",  # Select all data
    'outFields': "*",  # All fields
    'f': 'json',  # Return format
    # 'geometryType': 'esriGeometryPoint',  # Can change based on geometry type
    # 'outSR': '27700',  # Coordinate system. Unsure if needed, from documentation: If outSR is not specified, the geometry is returned in the spatial reference of the map.
    }
    
    url_geoservice = "https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Local_Nature_Reserves_England/FeatureServer/0/query"
    response_dataset = requests.get(url_geoservice, params=params)

    # CHeck request worked
    if response_dataset.status_code == 200:
        # Download geojson 
        with open(f"{temp_download_location}{dataset}.geojson", 'wb') as file:
            file.write(response_dataset.content)
        # Need to read geojson back in then write out the shapefile, as arcgis does not read geojson
        gdf = gpd.read_file(f"{temp_download_location}{dataset}.geojson")
        #Create folder if not already there
        if not os.path.exists(f"{download_location}{source}/{dataset}"):
            os.makedirs(f"{download_location}{source}/{dataset}")
        gdf.to_file(f"{download_location}{source}/{dataset}/{dataset}.shp")
    else:
        print(f"Request failed for {dataset}")

    # Download dataset details
    url_geoservice = "https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Local_Nature_Reserves_England/FeatureServer/0?f=json"
    response_details = requests.get(url_geoservice)
    if response_details.status_code == 200:
        details = response_details.json()
        print(details['editingInfo']['dataLastEditDate'])
    else:
        print(f"Request failed for details of {dataset}")

    

https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Local_Nature_Reserves_England/FeatureServer/0/query?


C:\Users\Lara\AppData\Local\Temp\ipykernel_29828\2540865707.py:28: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{download_location}{source}/{dataset}/{dataset}.shp")
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Area' to 'Shape__Are'
  ogr_write(
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Length' to 'Shape__Len'
  ogr_write(


1745507839750
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/National_Nature_Reserves_England/FeatureServer/0/query?


C:\Users\Lara\AppData\Local\Temp\ipykernel_29828\2540865707.py:28: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{download_location}{source}/{dataset}/{dataset}.shp")
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Area' to 'Shape__Are'
  ogr_write(
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Length' to 'Shape__Len'
  ogr_write(


1745507839750
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Priority_Habitats_Inventory_England/FeatureServer/0/query?


...

1745507839750
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/SSSI_Impact_Risk_Zones_England/FeatureServer/0/query?


C:\Users\Lara\AppData\Local\Temp\ipykernel_29828\2540865707.py:28: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{download_location}{source}/{dataset}/{dataset}.shp")
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Area' to 'Shape__Are'
  ogr_write(
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Length' to 'Shape__Len'
  ogr_write(


1745507839750
https://services-eu1.arcgis.com/WIfgdJeDbrZU1cnA/arcgis/rest/services/Ancient%20Tree%20Inventory%20(ATI)/FeatureServer/0/query?


C:\Users\Lara\AppData\Local\Temp\ipykernel_29828\2540865707.py:28: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{download_location}{source}/{dataset}/{dataset}.shp")
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Area' to 'Shape__Are'
  ogr_write(
c:\temp\venvs\data-downloads\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape__Length' to 'Shape__Len'
  ogr_write(


1745507839750
